In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import tensorflow_datasets as tfds
import os
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from Training_imagenette import GetDatasets
from Nets_original  import GetNeuralNetworkModel
from Stats import WeightQuantization, ActivationStats, CheckAccuracyAndLoss, QuantizationEffect, GetReadAndWrites
from FileAnalize import analize_file, analize_file_uno,analize_file_uno_ceros, save_file, load_file
from funciones import buffer_vectores
from datetime import datetime
import itertools
import pathlib
import os, sys
#from openpyxl import Workbook
#from openpyxl import load_workbook
from Simulation import buffer_simulation, save_obj, load_obj

tf.random.set_seed(1234)
np.random.seed(1234)

# 1) Training

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a) Get Dataset

Primero Obtendremos el dataset colorectal, especificando la distribucion de entrenamiento/validacion y testing (80%,5%,15% para este caso); las dimensiones de entrada de la imagen (227,227), el numero de clases (8) y el tamaño de los batches. 
Nota: Pueden aparecer un monton de mensajes de tensorflow durante la primera ejecucion, no impactan en nada

In [6]:
ds=tfds.load('imagenette/320px', split=['train','validation'], shuffle_files=True)

In [2]:
tfds.load('imagenette')

C:\Users\usuario\anaconda3\envs\env_first\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


{'train': <PrefetchDataset shapes: {image: (None, None, 3), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 'validation': <PrefetchDataset shapes: {image: (None, None, 3), label: ()}, types: {image: tf.uint8, label: tf.int64}>}

In [7]:
tf.data.experimental.enable_debug_mode()
trainBatchSize = testBatchSize = 1
_,_,test_dataset = GetDatasets('imagenette',(320,320), 10, trainBatchSize, testBatchSize)

<class 'list'>


ResourceExhaustedError: OOM when allocating tensor with shape[9469,500,375,3] and type uint8 on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:Pack] name: component_0

El resultado son datasets iterables como el siguiente:

In [3]:
train_set

<RepeatDataset shapes: ((None, 227, 227, 3), (None, 8)), types: (tf.float32, tf.float32)>

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; b) Get Model

Luego creamos la red, en principio no activaremos ni la cuantizacion ni el efecto de envejecimiento

In [1]:
AlexNet   = GetNeuralNetworkModel('AlexNet',(227,227,3),8, quantization = False, aging_active = False)
loss      = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
AlexNet.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

NameError: name 'GetNeuralNetworkModel' is not defined

In [4]:
AlexNet.evaluate(test_set)

75/75 [==============================] - 14s 185ms/step - loss: 2.0814 - accuracy: 0.1160


[2.081390857696533, 0.11599999666213989]

con el metodo summary() se puede ver los detalles de la red capa por capa, las capas Lambda emplean la cuantizacion y envejecimiento, las cuales estan presentes pero no activas cuando estas opciones estan desactivadas

In [ ]:
print('Las capas 0,3,9,11,17,19,25,31,37,40,45 y 50  contienen la informacion para su procesamiento')
print('Las capas 2,8,10,16,18,24,30,36,38,44,49 y 53 contienen las activaciones que son escritas en memoria')

In [5]:
AlexNet.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
lambda (Lambda)              (None, 227, 227, 3)       0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 227, 227, 3)       0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 55, 55, 96)        34944     
_________________________________________________________________
lambda_2 (Lambda)            (None, 55, 55, 96)        0         
_________________________________________________________________
re_lu (ReLU)                 (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 55, 96)        384   

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; c) Training

Si se utilizan pesos ya entrenados el entrenamiento se puede omitir

In [6]:
# Early Stopping
# --------------
#earlyStop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
#    
#AlexNet.fit(x=train_set,epochs=100,
#            steps_per_epoch  =int(np.ceil(train_size / train_batch_size)),
#            validation_data  =valid_set,
#            validation_steps =int(np.ceil(valid_size/ train_batch_size)), 
#            callbacks=[earlyStop])

a continuacion cargamos o guardamos los pesos en la ruta especificada usando el metodo load_weights/save_weights

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; d) Load/Save Weigths

In [4]:
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'AlexNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')
AlexNet.load_weights(wgt_dir)

OpError: NewRandomAccessFile failed to Create/Open: C:\Users\usuario\Desktop\CNN_Gating\Data\Trained Weights\AlexNet\Colorectal Dataset\Weights.data-00000-of-00001 : El sistema no puede encontrar el archivo especificado.
; No such file or directory

podemos evaluar el accuracy y loss de la red

In [14]:
(OrigLoss,OrigAcc) = AlexNet.evaluate(test_set)

750/750 [==============================] - 28s 37ms/step - loss: 0.3156 - accuracy: 0.8853


# 2) Stats

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Write/Read Stats

primero identificamos (manualmente) las capas procesadadas(Convoluciones, Full conectadas y Pooling) junto a las capas que contienen los resultados que se escribiran en el buffer (capas luego de la funcion de activacion y/o Normalizacion)

In [9]:
for index,layer in enumerate(AlexNet.layers):
    print(index,layer.name)
print('Las capas 0,3,9,11,17,19,25,31,37,40,45 y 50  contienen la informacion para su procesamiento')
print('Las capas 2,8,10,16,18,24,30,36,38,44,49 y 53 contienen las activaciones que son escritas en memoria')

0 input_1
1 lambda
2 lambda_1
3 Conv1
4 lambda_2
5 re_lu
6 batch_normalization
7 lambda_3
8 lambda_4
9 max_pooling2d
10 lambda_5
11 Conv2
12 lambda_6
13 re_lu_1
14 batch_normalization_1
15 lambda_7
16 lambda_8
17 max_pooling2d_1
18 lambda_9
19 Conv3
20 lambda_10
21 re_lu_2
22 batch_normalization_2
23 lambda_11
24 lambda_12
25 Conv4
26 lambda_13
27 re_lu_3
28 batch_normalization_3
29 lambda_14
30 lambda_15
31 Conv5
32 lambda_16
33 re_lu_4
34 batch_normalization_4
35 lambda_17
36 lambda_18
37 max_pooling2d_2
38 lambda_19
39 flatten
40 dense
41 lambda_20
42 re_lu_5
43 dropout
44 lambda_21
45 dense_1
46 lambda_22
47 re_lu_6
48 dropout_1
49 lambda_23
50 dense_2
51 lambda_24
52 tf.compat.v1.nn.softmax
53 lambda_25
Las capas 0,3,9,11,17,19,25,31,37,40,45 y 50  contienen la informacion para su procesamiento
Las capas 2,8,10,16,18,24,30,36,38,44,49 y 53 contienen las activaciones que son escritas en memoria


con el siguiente bloque obtenemos el numero de lecturas y escrituras por posicion de memoria tanto usando la estrategia de CNN Gating o sin usarla

In [1]:
num_address  = 290400  # Tamaño del buffer, igual a la capa mas grande de la red (este caso), o un tamaño fijo pre-establecido.
Indices      = [0,3,9,11,17,19,25,31,37,40,45,50] #Capas con la informacion de procesamiento 
samples      = 10 #Numero de imagenes
# Sin Power Gating:
Data         = GetReadAndWrites(AlexNet,Indices,num_address,samples,CNN_gating=False)
stats        = {'Lecturas': Data['Reads'],'Escrituras': Data['Writes']}
Baseline_Acceses   = pd.DataFrame(stats).reset_index(drop=False)
# Con Power Gating
Data     = GetReadAndWrites(AlexNet,Indices,num_address,samples,CNN_gating=True)
stats    = {'Lecturas': Data['Reads'],'Escrituras': Data['Writes']}
CNN_gating_Acceses = pd.DataFrame(stats).reset_index(drop=False)
#save_obj(Baseline_Acceses,'Data/Acceses/AlexNet/Colorectal Dataset/Baseline')
#save_obj(CNN_gating_Acceses,'Data/Acceses/AlexNet/CNN_gating_Adj')

NameError: name 'GetReadAndWrites' is not defined

Como resultado tenemos los siguientes dataframes

In [11]:
Baseline_Acceses

,index,Lecturas,Escrituras
0,0,687860,55
1,1,687860,55
2,2,687860,55
3,3,687860,55
4,4,687860,55
...,...,...,...
290395,290395,0,0
290396,290396,0,0
290397,290397,0,0
290398,290398,0,0


In [ ]:
Data

In [12]:
CNN_gating_Acceses

,index,Lecturas,Escrituras
0,0,129702,14
1,1,129702,14
2,2,129702,14
3,3,129718,14
4,4,129718,14
...,...,...,...
290395,290395,496,7
290396,290396,496,7
290397,290397,544,7
290398,290398,544,7


# 3) Quantization 

Ahora veremos como afecta la cuantizacion, tanto en pesos como en activaciones a la accuracy y loss

In [13]:
df = QuantizationEffect('AlexNet',test_set,wgt_dir,(227,227,3),8,test_batch_size)
#save_obj(df,'Data/Quantization/AlexNet/Colorectal Dataset/Quantization')

Activation fraction part
0  bits results:  acc:  0.35333332419395447 loss:  nan
1  bits results:  acc:  0.2933333218097687 loss:  10.372422218322754
2  bits results:  acc:  0.4586666524410248 loss:  6.999029159545898
3  bits results:  acc:  0.6466666460037231 loss:  3.6160128116607666
4  bits results:  acc:  0.8853333592414856 loss:  0.4703865051269531
5  bits results:  acc:  0.8853333592414856 loss:  0.4086773991584778
6  bits results:  acc:  0.8880000114440918 loss:  0.33622241020202637
7  bits results:  acc:  0.8866666555404663 loss:  0.3257526755332947
8  bits results:  acc:  0.8853333592414856 loss:  0.3284532129764557
9  bits results:  acc:  0.8840000033378601 loss:  0.31434592604637146
10  bits results:  acc:  0.8853333592414856 loss:  0.3158952295780182
11  bits results:  acc:  0.8853333592414856 loss:  0.3156990110874176
12  bits results:  acc:  0.8853333592414856 loss:  0.3155492842197418
13  bits results:  acc:  0.8853333592414856 loss:  0.3156721293926239
14  bits results: 

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a) Number of bits analysis

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; b) Used Config

En base a lo anterior parece prudente usar 11 bits de fraccion junto a 4 bits de parte entera tanto para pesos como activaciones, los resultados bajo esa configuracion se pueden obtener usando la funcion CheckAccurucyAndLoss()

In [16]:
CheckAccuracyAndLoss('AlexNet', test_set, wgt_dir, act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4, 
                    input_shape = (227,227,3), output_shape = 8, batch_size = test_batch_size);

aging_active en AddCustomLayers False
dentro de includin Aging False
dentro de GenerateAddressList 
aging_active dentro de aging_argiments False
dentro de Aging
aging_active dentro de la red Alexnet en capa 0 False
aging_active en AddCustomLayers False
aging_active en AddCustomLayers False
dentro de includin Aging False
dentro de GenerateAddressList 
aging_active dentro de aging_argiments False
dentro de Aging
aging_active dentro de la red Alexnet en capa 1 False
aging_active dentro de la red Alexnet en capa 2 False
aging_active en AddCustomLayers False
dentro de includin Aging False
dentro de GenerateAddressList 
aging_active dentro de aging_argiments False
dentro de Aging
aging_active dentro de la red Alexnet en capa 1 False
aging_active en AddCustomLayers False
aging_active en AddCustomLayers False
dentro de includin Aging False
dentro de GenerateAddressList 
aging_active dentro de aging_argiments False
dentro de Aging
aging_active en AddCustomLayers False
dentro de includin Aging F

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  c) Activation Stats

Por otro lado, para la configuracion anterior, veremos el valor medio,maximo,minimo y el ratio de saturacion tanto de las activaciones procesadas dentro de la unidad matricial de multiplicacion como de las almacenadas en el buffer. Nota: el ultimo parametro indica el numero de iteraciones que se deben realizar hasta agotar el dataset, se obtiene como numero de imagenes del dataset dividido en el batch size.

In [15]:
ActivationStats(AlexNet,test_set,11,4,24)

mean value (MMU): -0.18011768
mean value (Buffer): 0.13878344
maximum (MMU): 45.118935
minimum (MMU): -76.98705
maximum (Buffer): 44.172142
minimum (Buffer): -3.4607625
saturation ratio (MMU): 0.002840600757529724
saturation ratio (Buffer): 7.009283344612763e-06


# 3) Buffer Simulation

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a) Baseline

Ahora para el Baseline simularemos el comportamiento de 1 buffer durante la inferencia de 3 imagenes (solo 3 como ejemplo), la red se crea ahora activando la cuantizacion pero no el envejecimiento. LI y AI son los definidos en el item 2) Stats

In [16]:
train_batchSize = test_batchSize = 1
_,_,test_set = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, train_batchSize, test_batchSize)

#Obtain quantized network with the used configf
QAlexNet  = GetNeuralNetworkModel('AlexNet',(227,227,3),8, quantization = True, aging_active=False,
                                  word_size = 16, frac_size = 11)
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
metrics = ['accuracy']
QAlexNet.compile(optimizer=optimizer, loss=loss, metrics=metrics)
QAlexNet.load_weights(wgt_dir).expect_partial()
#Quantize the weights of the network too, with used config
WeightQuantization(model = QAlexNet, frac_bits = 11, int_bits = 4)

#Lista de capas acorde a 2.b
LI = [0,3,9,11,17,19,25,31,37,40,45,50]
AI = [2,8,10,16,18,24,30,36,38,44,49,53]
Buffer,ciclos =  buffer_simulation(QAlexNet, test_set, integer_bits = 4, fractional_bits = 11, samples = 3, start_from = 0,
                                  bit_invertion = False, bit_shifting = False, CNN_gating = False,
                                  buffer_size = 2*290400, write_mode ='default', save_results = False,
                                  results_dir = 'Data/Stats/AlexNet/Colorectal Dataset/CNN-Gated/',
                                  layer_indexes = LI , activation_indixes = AI)

buffer sections:  [0, 36300, 72600, 108900, 145200, 181500, 217800, 254100, 290400]
Simulation Started, time: 10:20:35 cycles:  0 offset:  0


CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:

El resultado es un diccionario como el siguiente donde por ejemplo Data contiene el ultimo valor registrado para cada celda de memoria.

In [ ]:
Buffer

In [ ]:
ciclos

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; b) CNN-Gated, buffer 2 MB

Se puede hacer lo mismo para distintas configuraciones de buffer y estrategias usadas

In [ ]:
#train_batchSize = test_batchSize = 1
#_,_,test_set = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, train_batchSize, test_batchSize)
#
##Obtain quantized network with the used configf
#QAlexNet  = GetNeuralNetworkModel('AlexNet',(227,227,3),8, quantization = True, aging_active=False,
#                                  word_size = 16, frac_size = 11)
#loss = tf.keras.losses.CategoricalCrossentropy()
#optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
#metrics = ['accuracy']
#QAlexNet.compile(optimizer=optimizer, loss=loss, metrics=metrics)
#QAlexNet.load_weights(wgtDir).expect_partial()
##Quantize the weights of the network too, with used config
#WeightQuantization(model = QAlexNet, frac_bits = 11, int_bits = 4)
#
##Lista de capas acorde a 2.b
#LI = [0,3,9,11,17,19,25,31,37,40,45,50]
#AI = [2,8,10,16,18,24,30,36,38,44,49,53]
#Buffer,ciclos =  buffer_simulation(QAlexNet, test_set, integer_bits = 4, fractional_bits = 11, samples = 150, start_from = 0,
#                                  bit_invertion = False, bit_shifting = False, CNN_gating = True,
#                                  buffer_size = 2*1024*1024, write_mode ='default', save_results = False,
#                                  results_dir = 'Data/Stats/AlexNet/Colorectal Dataset/CNN-Gated/',
#                                  layer_indexes = LI , activation_indixes = AI)

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; c) CNN-Gated, buffer ajustado a capa mas grande

In [ ]:
#train_batchSize = test_batchSize = 1
#_,_,test_set = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, train_batchSize, test_batchSize)
#
##Obtain quantized network with the used configf
#QAlexNet  = GetNeuralNetworkModel('AlexNet',(227,227,3),8, quantization = True, aging_active=False,
#                                  word_size = 16, frac_size = 11)
#loss = tf.keras.losses.CategoricalCrossentropy()
#optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
#metrics = ['accuracy']
#QAlexNet.compile(optimizer=optimizer, loss=loss, metrics=metrics)
#QAlexNet.load_weights(wgtDir).expect_partial()
##Quantize the weights of the network too, with used config
#WeightQuantization(model = QAlexNet, frac_bits = 11, int_bits = 4)
#
#
#LI = [0,3,9,11,17,19,25,31,37,40,45,50]
#AI = [2,8,10,16,18,24,30,36,38,44,49,53]
#Buffer,ciclos =  buffer_simulation(QAlexNet, test_set, integer_bits = 4, fractional_bits = 11, samples = 150, start_from = 0,
#                                  bit_invertion = False, bit_shifting = False, CNN_gating = True,
#                                  buffer_size = 2*290400, write_mode ='default', save_results = False,
#                                  results_dir = 'Data/Stats/AlexNet/Colorectal Dataset/CNN-Gated/',
#                                  layer_indexes = LI , activation_indixes = AI)

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ... d) para cargar los datos:

tambien puedes cargar los datos desde la ruta que especificaste para guardar los resultados (parametro results_dir + save_results de buffer_simulation())

In [ ]:
#Buffer  = load_obj('Data/Stats/AlexNet/Colorectal Dataset/CNN-Gated/Full Buffer/Buffer')
#ciclos  = load_obj('Data/Stats/AlexNet/Colorectal Dataset/CNN-Gated/Full Buffer/cycles')

# 4) Error Injection

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a) Measuring effect of faults in activations

por ultimo con este bloque vemos como se comporta el accuracy y loss frente a el envejecimiento de celdas.

In [18]:
from copy import deepcopy
from Stats import CheckAccuracyAndLoss
from Simulation import save_obj, load_obj
from datetime import datetime
import itertools
from Nets_original  import GetNeuralNetworkModel

#trainBatchSize = testBatchSize = 16 modificado por yamilka abajo
trainBatchSize = testBatchSize = 1
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, trainBatchSize, testBatchSize)


# Porciones del buffer con fallos a probar
Accs     = {0.00001:[],0.00005:[],0.0001:[]}
Loss     = {0.00001:[],0.00005:[],0.0001:[]}

# Tamaño del buffer (en bits)
network_size   = 290400*16
# Numero de muestras de distintas configuraciones de fallos a testear por cada valor de Accs/Loss
num_of_samples = 1
for Enumber in Accs:
    print(type(Accs))
    print(Enumber)
    n_bits_fails = np.ceil(Enumber*network_size).astype(int)    #numero de bits con fallos
    errors       = np.random.randint(0,2,n_bits_fails)          #tipo de fallos (0 o 1)
    # crear una representacion del buffer x indica celda inafectada, 1 celda con valor 1 permanente y 0 celda con valor 0 perm.
    buffer       = np.array(['x']*(network_size-n_bits_fails))  
    buffer       = np.concatenate([buffer,errors])
    print('tamaño del',len(buffer))
    
    for index in range(0,num_of_samples):
        np.random.shuffle(buffer)  # crear un orden aleatorio de los errores en el buffer
        # en las siguientes 4 lineas se obtienen las direcciones de los errores y los tipos de error
        #print(buffer)
        address_with_errors = np.reshape(buffer,(-1,16))
       # print(address_with_errors)
        #print(address_with_errors)
        address_with_errors = ["".join(i) for i in address_with_errors]
        #print(address_with_errors)
        #print(address_with_errors)
        error_mask = [y for x,y in enumerate(address_with_errors) if y.count('x') < 16]
        #print(error_mask)
        locs       = [x for x,y in enumerate(address_with_errors) if y.count('x') < 16]
        print('loooo',len(locs))
        print(len(error_mask))
        del address_with_errors
        #ahora se obtiene el loss y acc para esta configuracion.
        loss,acc   = CheckAccuracyAndLoss('AlexNet', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                            batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)
        print(loss)
        print(acc)
        Accs[Enumber].append(acc)
        Loss[Enumber].append(loss)
    print(str(Enumber)+' completada: ', datetime.now().strftime("%H:%M:%S"))
    #save_obj(Accs,'Data/Errors/AlexNet/Colorectal Dataset/Uniform distribution/Accs')
    #save_obj(Loss,'Data/Errors/AlexNet/Colorectal Dataset/Uniform distribution/Loss')

<class 'dict'>
1e-05
tamaño del 4646400
loooo 47
47
aging_active en AddCustomLayers False
dentro de includin Aging False
dentro de GenerateAddressList 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
aging_active dentro de aging_argiments False
dentro de Aging
aging_active dentro de la red Alexnet en capa 0 False
aging_active en AddCustomLayers False
aging_active en AddCustomLayers False
dentro de includin Aging False
dentro de GenerateAddressList 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
d

aging_active dentro de aging_argiments False
dentro de Aging
aging_active en AddCustomLayers False
aging_active en AddCustomLayers False
dentro de includin Aging False
dentro de GenerateAddressList 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de

aging_active en AddCustomLayers False
dentro de includin Aging False
dentro de GenerateAddressList 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask

El resultado es un diccionario con accuracy y loss para cada muestra, por ejemplo, el caso de 0.0001 del buffer con fallos la accuracy ronda entre el 71 y 76% en los 10 casos aleatorios probados.

In [19]:
Accs

{1e-05: [], 5e-05: [], 0.0001: []}

In [ ]:
Loss

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a) Measuring effect of faults in weights

Puedes hacer lo mismo para fallos en los pesos

In [9]:
from copy import deepcopy
from Stats import CheckAccuracyAndLoss
from Simulation import save_obj, load_obj
from datetime import datetime
import itertools


trainBatchSize = testBatchSize = 16
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, trainBatchSize, testBatchSize)


# Porciones del buffer con fallos a probar
Accs     = {0.00001:[],0.00005:[],0.0001:[]}
Loss     = {0.00001:[],0.00005:[],0.0001:[]}

# Tamaño del buffer (en bits)
network_size   = 885120*16
num_of_samples = 200
for Enumber in Accs:
    n_bits_fails = np.ceil(Enumber*290400).astype(int)
    errors       = np.random.randint(0,2,n_bits_fails)
    buffer       = np.array(['x']*(network_size-n_bits_fails))
    buffer       = np.concatenate([buffer,errors])
    for index in range(0,num_of_samples):
        np.random.shuffle(buffer)
        address_with_errors = np.reshape(buffer,(-1,16))
        address_with_errors = ["".join(i) for i in address_with_errors]
        error_mask = [y for x,y in enumerate(address_with_errors) if y.count('x') < 16]
        locs       = [x for x,y in enumerate(address_with_errors) if y.count('x') < 16]
        del address_with_errors
        loss,acc   = CheckAccuracyAndLoss('AlexNet', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                            batch_size=testBatchSize, verbose = 0, aging_active = False, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)
        print(acc)
        print(loss)
        print(index,' completados: ', datetime.now().strftime("%H:%M:%S"))
        Accs[Enumber].append(acc)
        Loss[Enumber].append(loss)
    print(str(Enumber)+' completada: ', datetime.now().strftime("%H:%M:%S"))
    #save_obj(Accs,'Data/Errors/AlexNet/Colorectal Dataset/Uniform distribution/weights Accs')
    #save_obj(Loss,'Data/Errors/AlexNet/Colorectal Dataset/Uniform distribution/weights Loss')

xxxxxxx0xxxxxxxx
xxxxxxxxxxx0xxxx
xxxxxxx1xxxxxxxx
xxxxxxx0xxxxxxxx
xxxxxxxxxxx0xxxx
xxxxxxx1xxxxxxxx
0.890666663646698
[289187, 503148, 542555]
0  completados:  12:03:04
xxxx0xxxxxxxxxxx
xxx0xxxxxxxxxxxx
xxxx1xxxxxxxxxxx
xxxx0xxxxxxxxxxx
xxx0xxxxxxxxxxxx
xxxx1xxxxxxxxxxx
0.890666663646698
[108713, 128139, 691189]
1  completados:  12:03:28
xxxxxxxxxx0xxxxx
0xxxxxxxxxxxxxxx
xxxxxxxxxxx1xxxx
xxxxxxxxxx0xxxxx
0xxxxxxxxxxxxxxx
xxxxxxxxxxx1xxxx
0.890666663646698
[40005, 342208, 640405]
2  completados:  12:03:50
xxxxxxxxxxxxxx1x
xxxxx0xxxxxxxxxx
xxxxx0xxxxxxxxxx
xxxxxxxxxxxxxx1x
xxxxx0xxxxxxxxxx
xxxxx0xxxxxxxxxx
0.890666663646698
[108143, 132892, 392936]
3  completados:  12:04:09
xxxxxxxxxxxx0xxx
1xxxxxxxxxxxxxxx
xxxxxxxx0xxxxxxx
xxxxxxxxxxxx0xxx
1xxxxxxxxxxxxxxx
xxxxxxxx0xxxxxxx
0.890666663646698
[73137, 165204, 396007]
4  completados:  12:04:27
xxxxxxx0xxxxxxxx
xxxxxxxxxxxxx1xx
xxxxxxxxxxxx0xxx
xxxxxxx0xxxxxxxx
xxxxxxxxxxxxx1xx
xxxxxxxxxxxx0xxx


KeyboardInterrupt: 